In [ ]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1. Представление и предобработка текстовых данных 

1.1 Операции по предобработке:
* токенизация
* стемминг / лемматизация
* удаление стоп-слов
* удаление пунктуации
* приведение к нижнему регистру
* любые другие операции над текстом

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer

In [ ]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

Реализовать функцию `preprocess_text(text: str)`, которая:
* приводит строку к нижнему регистру
* заменяет все символы, кроме a-z, A-Z и знаков .,!? на пробел


In [ ]:
def preprocess_text(text):
  text = text.lower()
  text = re.sub(r"([.,!?])", r" \1 ", text)
  text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
  return text

In [ ]:
preprocess_text(text)

'select your preferences and run the install command . stable represents the most currently tested and supported version of pytorch . note that libtorch is only available for c '

1.2 Представление текстовых данных при помощи бинарного кодирования


Представить первое предложение из `text` в виде тензора `sentence_t`: `sentence_t[i] == 1`, если __слово__ с индексом `i` присуствует в предложении.

In [ ]:
from string import punctuation
def clean_words(input_str):
  word_list = input_str.lower().replace('\n',' ').split()
  word_list = [word.strip(punctuation) for word in word_list]
  return word_list

In [ ]:
text_words = clean_words(text)
# print(text_words)
possible_words = list(set(text_words))
#print(possible_words)
word_t = torch.zeros(len(text_words), len(text_words))
for idx, word in enumerate(text_words):
  #print(idx, word)
  word_t[idx, text_words.index(word)] = 1
word_t.shape

torch.Size([27, 27])

In [ ]:
word_t

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0

## 2. Классификация фамилий по национальности

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`. 

2.2 Закодировать национальности числами, начиная с 0.

2.3 Разбить датасет на обучающую и тестовую выборку

2.4 Реализовать класс `Vocab` (токен = __символ__)

2.5 Реализовать класс `SurnamesDataset`

2.6. Обучить классификатор.

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

Решим первые три пункта с помощью склерн:

In [ ]:
train_surnames = pd.read_csv('surnames.csv')
class_encoder = LabelEncoder()
train_surnames['nationality'] = class_encoder.fit_transform(train_surnames['nationality'])
train_surnames.head(5)

,surname,nationality
0,Woodford,4
1,Coté,5
2,Kore,4
3,Koury,0
4,Lebzak,14


In [ ]:
X = train_surnames['surname'].str.lower()
y = train_surnames['nationality']
n_classes = y.nunique()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
class Vocab:
  def __init__(self, data):
    tokens = set()
    for item in data:
      tokens.update(item)

    self.idx_to_token = dict(enumerate(tokens))
    self.token_to_idx = {token: idx for idx, token in self.idx_to_token.items()}
    self.vocab_len = len(self.idx_to_token)

In [ ]:
class SurnamesDataset(Dataset):
  def __init__(self, X, y, vocab: Vocab):
    self.X = X
    self.y = y
    self.vocab = vocab

  def vectorize(self, surname):
    '''Генерирует представление фамилии surname в при помощи бинарного кодирования (см. 1.2)'''
    surname_t = torch.zeros(self.vocab.vocab_len)
    for token in surname:
      surname_t[self.vocab.token_to_idx[token]] = 1
    return surname_t
    
  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return self.vectorize(self.X.iloc[idx]), self.y.iloc[idx]

In [ ]:
vocab = Vocab(X) #  , level='char')
train_dataset = SurnamesDataset(X_train, y_train, vocab)
test_dataset = SurnamesDataset(X_test, y_test, vocab)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [ ]:
model = nn.Sequential(nn.Linear(vocab.vocab_len, 300),
        nn.ReLU(),
        nn.Linear(300, n_classes)).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
n_epochs = 51
for epoch in range(n_epochs):
  epoch_loss = 0
  for X_batch, y_batch in train_loader:
    predictions = model(X_batch.cuda())
    loss = criterion(predictions, y_batch.cuda())
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    epoch_loss += loss.item()
  
  with torch.no_grad():
    val_loss, val_acc = 0, 0
    for X_batch, y_batch in test_loader:
      predictions = model(X_batch.cuda())
      loss = criterion(predictions, y_batch.cuda()).item()
      acc = accuracy_score(y_batch, predictions.argmax(dim=1).cpu().detach()).item()
      val_loss += loss
      val_acc += acc
    if epoch % 5 == 0:
      print(f'#{epoch} Training loss: {epoch_loss / len(train_loader)} val_loss: {val_loss / len(test_loader)}')

#0 Training loss: 2.2145495725714643 val_loss: 1.9049338234795465
#5 Training loss: 1.4165997626124949 val_loss: 1.4279339181052313
#10 Training loss: 1.287342918091926 val_loss: 1.3409534295399983
#15 Training loss: 1.208338799683944 val_loss: 1.2856463723712497
#20 Training loss: 1.1499089667762534 val_loss: 1.2134700417518616
#25 Training loss: 1.1049132744471233 val_loss: 1.221351510948605
#30 Training loss: 1.0656294658564138 val_loss: 1.2161329521073236
#35 Training loss: 1.0329764735871467 val_loss: 1.151521106561025
#40 Training loss: 1.0056504598562268 val_loss: 1.16902337802781
#45 Training loss: 0.9779457423997961 val_loss: 1.1723580559094746
#50 Training loss: 0.9572677258132161 val_loss: 1.171672781308492


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)
X_batch, y_batch = next(iter(test_loader))
predictions = model(X_batch.cuda()).argmax(dim=1).cpu().detach()
print(classification_report(y_batch, predictions))

              precision    recall  f1-score   support

           0       0.83      1.00      0.91       346
           1       0.48      0.61      0.54        36
           2       0.43      0.12      0.19        81
           3       0.50      0.14      0.22        49
           4       0.58      0.80      0.67       567
           5       0.00      0.00      0.00        36
           6       0.45      0.34      0.39       118
           7       0.57      0.25      0.35        32
           8       0.69      0.22      0.33        41
           9       0.47      0.44      0.45       108
          10       0.71      0.57      0.63       161
          11       0.36      0.27      0.31        15
          12       0.56      0.40      0.47        25
          13       0.00      0.00      0.00        14
          14       0.74      0.79      0.76       482
          15       0.00      0.00      0.00        13
          16       0.62      0.26      0.37        57
          17       0.00    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
surname = 'kalashnikov'
x = train_dataset.vectorize(surname).unsqueeze(0).cuda()
predictions = model(x).argmax(dim=1).cpu().detach()
class_encoder.inverse_transform(predictions)

array(['Russian'], dtype=object)

In [ ]:
_, predictions = model(x).topk(k=3, dim=1)
class_encoder.inverse_transform(predictions.cpu().detach().squeeze())

array(['Russian', 'Czech', 'Greek'], dtype=object)

## 3. Классификация обзоров ресторанов

Датасет: https://disk.yandex.ru/d/nY1o70JtAuYa8g

3.1 Считать файл `yelp/raw_train.csv`. Оставить от исходного датасета 10% строчек.

3.2 Воспользоваться функцией `preprocess_text` из 1.1 для обработки текста отзыва. Закодировать рейтинг числами, начиная с 0.

3.3 Разбить датасет на обучающую и тестовую выборку

3.4 Реализовать класс `Vocab` (токен = слово)

3.5 Реализовать класс `ReviewDataset`

3.6 Обучить классификатор

3.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)


In [ ]:
class Vocab:
  def __init__(self, data):
    self.idx_to_token = ...
    self.token_to_idx = ...
    self.vocab_len = ...

In [ ]:
class ReviewDataset(Dataset):
  def __init__(self, X, y, vocab: Vocab):
    self.X = X
    self.y = y
    self.vocab = vocab

  def vectorize(self, review):
    '''Генерирует представление отзыва review при помощи бинарного кодирования (см. 1.2)'''
    
  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return ...